# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
if Path.cwd().parts[-1] != 'Multifirefly-Project':
    if Path.cwd().parts[-1] != 'notebooks':
        os.chdir('..')
    from add_path import find_path
    current_path = find_path()
    os.chdir(current_path)

import sys
from data_wrangling import basic_func, process_raw_data
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization import plot_behaviors_utils
from non_behavioral_analysis.neural_data_analysis import neural_data_processing, visualize_neural_data
from non_behavioral_analysis.neural_data_analysis.neural_vs_behavioral import prep_monkey_data, prep_monkey_data, prep_monkey_data, prep_target_data, neural_vs_behavioral_class, neural_data_modeling, reduce_multicollinearity
from non_behavioral_analysis.neural_data_analysis.planning_neural import planning_neural_class

import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import math
import seaborn as sns
from matplotlib import rc
from os.path import exists
from numpy import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from importlib import reload
import neo
import gc

import sys
import os
import numpy as np
import torch
from numpy import pi
from matplotlib import pyplot as plt

import numpy as np
import scipy.linalg as linalg
import scipy.interpolate as interpolate
from scipy.signal import fftconvolve


from scipy.io import loadmat
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import sys

import scipy.sparse as sparse
import matplotlib.pylab as plt
import subprocess

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

Changed the directory to 'Multifirefly-Project'.
Added /Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods to the path.
done
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [ ]:
data_item = neural_vs_behavioral_class.NeuralVsBehavioralClass(raw_data_folder_path=raw_data_folder_path)
data_item.streamline_preparing_neural_and_behavioral_data()

Retrieved monkey data from  all_monkey_data/processed_data/monkey_Schro/data_0416/monkey_information.csv  and ff data from  all_monkey_data/processed_data/monkey_Schro/data_0416/ff_basic_info.npz
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from  all_monkey_data/processed_data/monkey_Schro/data_0416/ff_dataframe.h5
When take out monkey subset for GUAT, 139 clusters out of 241 are too close to the target or the last target. Those clusters are filtered out.
The number of new trials that are used to separate stop clusters is 470
Retrieved all_trial_patterns
Retrieved pattern_frequencies
Retrieved all_trial_features
Retrieved feature_statistics
Retrieved scatter_around_target_df


Neural data (changed bin_width here)

# Planning

## get data

In [ ]:
spike_df = neural_data_processing.make_spike_df(raw_data_folder_path="all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330")
bin_width=0.1
time_bins, all_binned_spikes = neural_data_processing.bin_spikes(spike_df, bin_width=bin_width)
neural_bins_df = pd.DataFrame(all_binned_spikes, columns=np.arange(all_binned_spikes.shape[1]))
neural_bins_df['bin'] = np.arange(neural_bins_df.shape[0])

num_bins = all_binned_spikes.shape[0]
unique_clusters = np.sort(spike_df.cluster.unique())

In [ ]:
ref_point_mode='distance'
ref_point_value=-150

normalize = False
eliminate_outliers = False
use_curvature_to_ff_center = False
curv_of_traj_mode = 'time'
window_for_curv_of_traj=[-1, 1]

pn = planning_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path)
pn.streamline_organizing_info(ref_point_mode='time after stop ff visible', ref_point_value=0.1, 
                               curv_of_traj_mode=curv_of_traj_mode, window_for_curv_of_traj=window_for_curv_of_traj, truncate_curv_of_traj_by_time_of_capture=True,
                               use_curvature_to_ff_center=use_curvature_to_ff_center,  eliminate_outliers=eliminate_outliers)


/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods/non_behavioral_analysis/eye_positions.py:119: RuntimeWarning: divide by zero encountered in divide
  numerator_component = 1/np.tan(ver_theta)**2 - np.tan(inside_tan)**2


Retrieved monkey data from  all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0330/processed_data/basic_info/monkey_information.csv  and ff data from  all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0330/processed_data/basic_info/ff_basic_info.npz


/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods/non_behavioral_analysis/eye_positions.py:119: RuntimeWarning: divide by zero encountered in divide
  numerator_component = 1/np.tan(ver_theta)**2 - np.tan(inside_tan)**2


shared_stops_near_ff_df has 528 rows
Retrieving shared_stops_near_ff_df succeeded
Number of new ff xy within 1 cm of monkey: 1 out of 299. Setting them to monkey position.
Note: 1 fireflies are too close to the monkey/agent. Their angles are set to 0.
Retrieving Bruno_stop_0_1 from all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0330/processed_data/updated_planning_info/heading_info/test/Bruno_stop_0_1 succeeded


In [ ]:
columns_to_drop = ['alt_opt_arc_dheading',
 'alt_arc_curv',
  'angle_from_stop_ff_landing_to_alt_ff',
  
 # preserve "alt + alt" or 'stop + stop' but not 'alt + stop' or 'stop + alt'. Otherwise the correlation shall be too great
#  'alt_ff_distance_when_stop_ff_first_seen',
#  'alt_ff_distance_when_stop_ff_last_seen',
#  'stop_ff_distance_when_alt_ff_first_seen',
#  'stop_ff_distance_when_alt_ff_last_seen',
#  'stop_ff_angle_when_alt_ff_first_seen',
#  'alt_ff_angle_when_stop_ff_first_seen',
#  'stop_ff_angle_when_alt_ff_last_seen',
#  'alt_ff_angle_when_stop_ff_last_seen',
]


In [ ]:
gc.collect()

0

In [ ]:
df = pn.both_ff_across_time_df
df

,alt_ff_angle,alt_ff_distance,alt_arc_curv,alt_opt_arc_curv,alt_opt_arc_dheading,time_rel_to_stop,traj_curv,stop_ff_angle,stop_ff_distance,stop_arc_curv,stop_opt_arc_curv,stop_opt_arc_dheading,stop_point_index,bin
point_index,,,,,,,,,,,,,,
1833,-0.54674,398.54102,-0.00261,-0.00250,-1.04335,-1.82635,-0.00120,-0.48219,239.04897,-0.00388,-0.00357,-0.88072,1945,122
1848,-0.26897,359.67684,-0.00148,-0.00163,-0.58415,-1.57759,-0.00644,-0.22692,199.34043,-0.00226,-0.00275,-0.53951,1945,123
1864,-0.03726,308.13876,-0.00024,-0.00045,-0.13877,-1.31197,-0.00315,-0.00303,147.57340,-0.00004,-0.00096,-0.14025,1945,124
1879,0.01101,267.40960,0.00008,0.00036,0.09646,-1.06297,0.00117,0.06069,106.89807,0.00113,0.00117,0.11313,1945,125
1894,-0.00557,267.22771,-0.00004,-0.00000,-0.00026,-0.81405,0.00117,0.04452,106.71935,0.00083,0.00117,0.11299,1945,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19425,0.02174,110.11885,0.00039,0.00100,0.10074,1.57990,0.00100,NaN,NaN,NaN,NaN,NaN,19328,1278
19440,0.03977,60.23512,0.00132,0.00067,0.03392,1.82890,0.00067,NaN,NaN,NaN,NaN,NaN,19328,1279
19455,0.03747,24.60177,0.00234,0.00234,0.03417,2.07790,0.00234,NaN,NaN,NaN,NaN,NaN,19328,1280


In [ ]:
all_planning_info = pn.get_all_planning_info(time_bins, both_ff_across_time_df_exists_ok=False)

# drop rows with NA in all_planning_info and print the percentage of rows dropped
all_planning_info2 = all_planning_info.drop(columns=columns_to_drop).copy()
all_planning_info2 = all_planning_info2.dropna().copy()
all_planning_info2.drop(columns={'stop_point_index', 'point_index'}, inplace=True)
print("Percentage of rows dropped: ", 1 - all_planning_info2.shape[0] / len(all_planning_info))
data_item.final_behavioral_data = all_planning_info2.copy()

neural_bins_sub2 = neural_bins_df[neural_bins_df['bin'].isin(all_planning_info2['bin'].values)]
x_var = neural_bins_sub2.drop(columns=['bin']).values

Having processed 0 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 10 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 20 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 30 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 40 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 50 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 60 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 70 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 80 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 90 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 100 rows out of 299 of the stops_near_ff_df for both_ff_across_time_df.
Having processed 110 rows out of

In [ ]:
max_bin = data_item.final_behavioral_data['bin'].max()
binned_spikes_matrix, binned_spikes_df = neural_data_processing.prepare_binned_spikes_matrix_and_df(max_bin, x_var)
x_var = binned_spikes_matrix

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.DataFrame(all_planning_info.isnull().sum())

,0
point_index,0
alt_ff_angle,156
alt_ff_distance,156
alt_arc_curv,156
alt_opt_arc_curv,156
alt_opt_arc_dheading,156
time_rel_to_stop,156
traj_curv,156
stop_ff_angle,1713
stop_ff_distance,1713


In [ ]:
print('x_var.shape:', x_var.shape)
print('binned_spikes_df.shape:', binned_spikes_df.shape)
print('data_item.final_behavioral_data.shape:', data_item.final_behavioral_data.shape)

x_var.shape: (2156, 16)
spikes_in_bins_df.shape: (2156, 17)
final_behavioral_data.shape: (2156, 13)


In [ ]:
# conduct linear regression on X and y
plt.rcParams["figure.figsize"] = (20, 10)
bins_to_plot = range(all_planning_info.bin.max())
for i, column in enumerate(data_item.final_behavioral_data.columns):
    visualize_neural_data.plot_regression(data_item.final_behavioral_data, column, x_var, bins_to_plot=None, min_R_squared_to_plot=0.3)
    # if i == 3:
    #     break

R: 0.08, R^2: 0.006 -- alt_ff_angle
R: 0.1, R^2: 0.011 -- alt_ff_distance
R: 0.07, R^2: 0.005 -- alt_opt_arc_curv
R: 0.1, R^2: 0.01 -- time_rel_to_stop
R: 0.09, R^2: 0.009 -- traj_curv
R: 0.08, R^2: 0.007 -- stop_ff_angle
R: 0.1, R^2: 0.011 -- stop_ff_distance
R: 0.08, R^2: 0.007 -- stop_arc_curv
R: 0.09, R^2: 0.009 -- stop_opt_arc_curv
R: 0.09, R^2: 0.008 -- stop_opt_arc_dheading
R: 0.37, R^2: 0.135 -- bin
R: 0.1, R^2: 0.009 -- angle_from_m_before_stop_to_stop_ff
R: 0.05, R^2: 0.003 -- angle_from_m_before_stop_to_alt_ff


### VIF

In [ ]:
vif = pd.DataFrame({
    "vif": [variance_inflation_factor(data_item.final_behavioral_data.values, i) for i in range(data_item.final_behavioral_data.shape[1])],
    "features": data_item.final_behavioral_data.columns
}).sort_values(by='vif', ascending=False).round(1)
vif.reset_index(drop=False, inplace=True)

vif

,index,VIF Factor,features
0,35,10094.00000,avg_target_last_seen_angle_to_boundary
1,34,9992.10000,avg_target_last_seen_angle
2,30,9895.00000,avg_target_angle
3,31,9740.90000,avg_target_angle_to_boundary
4,42,1006.00000,avg_target_cluster_last_seen_angle_to_boundary
5,41,778.90000,avg_target_cluster_last_seen_angle
6,40,311.70000,avg_target_cluster_last_seen_distance
7,4,269.20000,min_abs_ff_angle
8,5,257.70000,min_abs_ff_angle_boundary
9,33,248.70000,avg_target_last_seen_distance


In [ ]:
specific_columns = vif.iloc[:15].features.values
specific_columns

In [ ]:
fff = [feature for feature in vif.features.tolist() if ('alt' in feature) and ('stop' in feature)]
fff

In [ ]:
fff = [feature for feature in vif.features.tolist() if ('alt_arc_' in feature) or ('alt_opt' in feature)]
fff

In [ ]:
 specific_columns = ['stop_ff_distance',
 'stop_ff_angle',
 'stop_opt_arc_dheading',
 'stop_arc_curv']

 specific_columns = ['stop_ff_angle',
 'stop_ff_distance',
 'stop_opt_arc_dheading',
 'stop_opt_arc_curv',
 'stop_arc_curv']

In [ ]:
 specific_columns = ['alt_ff_angle',
 'alt_ff_distance',
 'alt_opt_arc_dheading',
 'alt_opt_arc_curv',
 'alt_arc_curv']

In [ ]:
fff = [feature for feature in vif.features.tolist() if ('alt' in feature) & ('first_seen' not in feature) & ('last_seen' in feature)]
fff

### Correlations between features

In [ ]:
# calculate the correlation coefficient among the columns with VIF > 5
# specific_columns = vif[vif["vif"] > 5].features.values
corr_coeff = data_item.final_behavioral_data[specific_columns].corr()
#plt.figure(figsize = (6, 6))
plt.figure(figsize = (12, 12))
sns.heatmap(corr_coeff, cmap='coolwarm', annot=True, linewidths=1, vmin=-1)
plt.show()

## Categorize variables

In [ ]:
data_item.final_behavioral_data.columns

In [ ]:
temporal_vars = ['time_rel_to_stop',
 'time_when_alt_ff_first_seen_rel_to_stop',
 'time_when_stop_ff_first_seen_rel_to_stop',
 'time_when_alt_ff_last_seen_rel_to_stop',
 'time_when_stop_ff_last_seen_rel_to_stop',
 ]

spatial_vars = [x for x in data_item.final_behavioral_data.columns if x not in temporal_vars]
spatial_vars